# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 28 2022 2:51PM,253945,10,0086271414,ISDIN Corporation,Released
1,Dec 28 2022 2:51PM,253944,10,SNAP1228221,"Snap Medical Industries, LLC",Released
2,Dec 28 2022 2:45PM,253943,10,SNAP33787665,"Snap Medical Industries, LLC",Released
3,Dec 28 2022 2:45PM,253943,10,SNAPJ2387287,"Snap Medical Industries, LLC",Released
4,Dec 28 2022 2:45PM,253943,10,SNAPK1102663257,"Snap Medical Industries, LLC",Released
5,Dec 28 2022 2:43PM,253939,10,0086271415,ISDIN Corporation,Released
6,Dec 28 2022 2:43PM,253939,10,0086271536,ISDIN Corporation,Released
7,Dec 28 2022 2:43PM,253939,10,0086271537,ISDIN Corporation,Released
8,Dec 28 2022 2:43PM,253939,10,0086271396,ISDIN Corporation,Released
9,Dec 28 2022 2:43PM,253939,10,0086271398,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,253939,Released,23
41,253942,Released,1
42,253943,Released,3
43,253944,Released,1
44,253945,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Released
id,,
253939,NaN,23.0
253942,NaN,1.0
253943,NaN,3.0
253944,NaN,1.0
253945,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Released
id,,
253855,0.0,5.0
253856,8.0,4.0
253862,0.0,1.0
253863,0.0,1.0
253865,0.0,38.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,Completed,Released
id,,
253855,0,5
253856,8,4
253862,0,1
253863,0,1
253865,0,38


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Released
0,253855,0,5
1,253856,8,4
2,253862,0,1
3,253863,0,1
4,253865,0,38


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,id,Completed,Released
0,253855,,5
1,253856,8,4
2,253862,,1
3,253863,,1
4,253865,,38


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 28 2022 2:51PM,253945,10,ISDIN Corporation
1,Dec 28 2022 2:51PM,253944,10,"Snap Medical Industries, LLC"
2,Dec 28 2022 2:45PM,253943,10,"Snap Medical Industries, LLC"
5,Dec 28 2022 2:43PM,253939,10,ISDIN Corporation
28,Dec 28 2022 2:42PM,253942,16,Sartorius Stedim North America
29,Dec 28 2022 2:36PM,253938,19,"Avet Pharmaceuticals Labs, Inc"
30,Dec 28 2022 2:27PM,253937,19,"Avet Pharmaceuticals Labs, Inc"
31,Dec 28 2022 2:23PM,253936,10,"ClearSpec, LLC"
32,Dec 28 2022 2:17PM,253935,19,ACG North America LLC
33,Dec 28 2022 1:57PM,253934,19,"NAPP Technologies, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Released
0,Dec 28 2022 2:51PM,253945,10,ISDIN Corporation,,1
1,Dec 28 2022 2:51PM,253944,10,"Snap Medical Industries, LLC",,1
2,Dec 28 2022 2:45PM,253943,10,"Snap Medical Industries, LLC",,3
3,Dec 28 2022 2:43PM,253939,10,ISDIN Corporation,,23
4,Dec 28 2022 2:42PM,253942,16,Sartorius Stedim North America,,1
5,Dec 28 2022 2:36PM,253938,19,"Avet Pharmaceuticals Labs, Inc",,1
6,Dec 28 2022 2:27PM,253937,19,"Avet Pharmaceuticals Labs, Inc",,1
7,Dec 28 2022 2:23PM,253936,10,"ClearSpec, LLC",,1
8,Dec 28 2022 2:17PM,253935,19,ACG North America LLC,,1
9,Dec 28 2022 1:57PM,253934,19,"NAPP Technologies, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Completed
0,Dec 28 2022 2:51PM,253945,10,ISDIN Corporation,1,
1,Dec 28 2022 2:51PM,253944,10,"Snap Medical Industries, LLC",1,
2,Dec 28 2022 2:45PM,253943,10,"Snap Medical Industries, LLC",3,
3,Dec 28 2022 2:43PM,253939,10,ISDIN Corporation,23,
4,Dec 28 2022 2:42PM,253942,16,Sartorius Stedim North America,1,
5,Dec 28 2022 2:36PM,253938,19,"Avet Pharmaceuticals Labs, Inc",1,
6,Dec 28 2022 2:27PM,253937,19,"Avet Pharmaceuticals Labs, Inc",1,
7,Dec 28 2022 2:23PM,253936,10,"ClearSpec, LLC",1,
8,Dec 28 2022 2:17PM,253935,19,ACG North America LLC,1,
9,Dec 28 2022 1:57PM,253934,19,"NAPP Technologies, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Dec 28 2022 2:51PM,253945,10,ISDIN Corporation,1,
1,Dec 28 2022 2:51PM,253944,10,"Snap Medical Industries, LLC",1,
2,Dec 28 2022 2:45PM,253943,10,"Snap Medical Industries, LLC",3,
3,Dec 28 2022 2:43PM,253939,10,ISDIN Corporation,23,
4,Dec 28 2022 2:42PM,253942,16,Sartorius Stedim North America,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet


,Date,id,Warehouse,Customer,Released,Completed
0,Dec 28 2022 2:51PM,253945,10,ISDIN Corporation,1,NaN
1,Dec 28 2022 2:51PM,253944,10,"Snap Medical Industries, LLC",1,NaN
2,Dec 28 2022 2:45PM,253943,10,"Snap Medical Industries, LLC",3,NaN
3,Dec 28 2022 2:43PM,253939,10,ISDIN Corporation,23,NaN
4,Dec 28 2022 2:42PM,253942,16,Sartorius Stedim North America,1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Dec 28 2022 2:51PM,253945,10,ISDIN Corporation,1,0.0
1,Dec 28 2022 2:51PM,253944,10,"Snap Medical Industries, LLC",1,0.0
2,Dec 28 2022 2:45PM,253943,10,"Snap Medical Industries, LLC",3,0.0
3,Dec 28 2022 2:43PM,253939,10,ISDIN Corporation,23,0.0
4,Dec 28 2022 2:42PM,253942,16,Sartorius Stedim North America,1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Completed
Warehouse,,,
10,8632799,293,0.0
16,253942,1,0.0
19,2285165,34,8.0
21,253916,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Completed
0,10,8632799,293,0.0
1,16,253942,1,0.0
2,19,2285165,34,8.0
3,21,253916,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Completed
0,10,293,0.0
1,16,1,0.0
2,19,34,8.0
3,21,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,293.0
1,16,Released,1.0
2,19,Released,34.0
3,21,Released,1.0
4,10,Completed,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,21
Status,,,,
Completed,0.0,0.0,8.0,0.0
Released,293.0,1.0,34.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,21
0,Completed,0.0,0.0,8.0,0.0
1,Released,293.0,1.0,34.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,21
0,Completed,0.0,0.0,8.0,0.0
1,Released,293.0,1.0,34.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()